In [ ]:
install.packages("remotes")
install.packages("devtools")
system("apt install libgsl-dev", intern=TRUE)
devtools::install_github("paulponcet/lplyr")

In [ ]:
if (!requireNamespace("BiocManager", quietly = TRUE))
    install.packages("BiocManager")
BiocManager::install(version = '3.19',ask = FALSE)

The following step can take about 15 min to install.

In [ ]:
BiocManager::install("clusterProfiler")
BiocManager::install("enrichplot")
BiocManager::install("ggplot2")
BiocManager::install("msigdbr")
BiocManager::install("dplyr")
BiocManager::install("DOSE")
BiocManager::install("forcats")
BiocManager::install("AnnotationDbi")
BiocManager::install("org.Hs.eg.db")

In [ ]:
library(clusterProfiler)
library(enrichplot)
library(ggplot2)
library(msigdbr)
library(dplyr)
library(lplyr)
library(DOSE)
library(forcats)
library(AnnotationDbi)
library(org.Hs.eg.db)

In [ ]:
download.file("https://raw.githubusercontent.com/10XGenomics/analysis_guides/main/Visium_HD_GSEA/All_gene_exp_ROI.csv", "All_gene_exp_ROI.csv")
df <- read.csv("All_gene_exp_ROI.csv", header = TRUE)
head(df)

In [ ]:
entrez_data <- AnnotationDbi::select(org.Hs.eg.db, keys = df$SYMBOL,columns = c("SYMBOL", "ENTREZID"),keytype = "SYMBOL")
head(entrez_data)

In [ ]:
anno_result <- entrez_data %>%
  filter(!is.na(ENTREZID)) %>%
  inner_join(df, by = "SYMBOL",relationship = "many-to-many")

In [ ]:
head(anno_result)

In [ ]:
geneList <- with(anno_result, setNames(Log2FC, ENTREZID))
geneList <- sort(geneList, decreasing = TRUE)

In [ ]:
go_t2g <- msigdbr(species = "Homo sapiens", category = "C5", subcategory = "GO:MF") %>%
  dplyr::select(gs_name, entrez_gene)
head(go_t2g)

In [ ]:
go <- GSEA(geneList, TERM2GENE = go_t2g)
go

In [ ]:
go <- setReadable(go, 'org.Hs.eg.db', 'ENTREZID')

In [ ]:
#set plot size
options(repr.plot.width=16, repr.plot.height=16)

#basic dotplot
dotplot(go, showCategory=10, split=".sign") + facet_grid(.~.sign)

#save a publicaiton-quality version of the plot
ggsave("go_dotplot.pdf",
        dpi = 600,
        width = 35, height = 25, unit = "cm")

In [ ]:
ggplot(go, showCategory=40,
       aes(NES, fct_reorder(Description, NES),
           fill=p.adjust)) +
  geom_col() +
  scale_fill_gradientn(colours=c("#b3eebe",
                                 "#46bac2", "#371ea3"),
                       guide=guide_colorbar(reverse=TRUE)) +
  theme_dose(12) +
  xlab("Normalized Enrichment Score") +
  ylab(NULL) +
  ggtitle("GO: Molecular Function")

In [ ]:
#gsea plot
gseaplot(go, by = "all", title = "DNA Replication Origin Binding", geneSetID = 38)

In [ ]:
# cnet plot for visualizing gene networks
cnetplot(go, categorySize="pvalue", color.params = list(foldChange=geneList), showCategory = c("GOMF_DNA_REPLICATION_ORIGIN_BINDING"))

In [ ]:
c4_t2g <- msigdbr(species = "Homo sapiens", category = "C4", subcategory = "CGN") %>%
  dplyr::select(gs_name, entrez_gene)
c4 <- GSEA(geneList, TERM2GENE = c4_t2g)

c4 <- setReadable(c4, 'org.Hs.eg.db', 'ENTREZID')

#plots
ggplot(c4, showCategory=40,
       aes(NES, fct_reorder(Description, NES),
           fill=p.adjust)) +
  geom_col() +
  scale_fill_gradientn(colours=c("#b3eebe",
                                 "#46bac2", "#371ea3"),
                       guide=guide_colorbar(reverse=TRUE)) +
  theme_dose(12) +
  xlab("Normalized Enrichment Score") +
  ylab(NULL) +
  ggtitle("Cancer Gene Networks")

In [ ]:
gseaplot(c4, by = "all", title = "PECAM1 Network", geneSetID = "GNF2_PECAM1")
cnetplot(c4, categorySize="pvalue", color.params = list(foldChange=geneList), showCategory = c("GNF2_PECAM1"))

In [ ]:
sessionInfo()